https://github.com/danielgatis/rembg

In [188]:
import os

os.chdir('../1')

In [197]:
from rembg import remove
import cv2

input_path = '1.jpg'

input = cv2.imread(input_path)
output = remove(input)
gray = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 1, 255, 0)[1]
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
area = 0

for cnt in contours:
    area += cv2.contourArea(cnt)

cv2.drawContours(output, contours, -1, (0, 255, 0), 3)

cv2.imshow('Contours', output)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [198]:
from skimage.filters import threshold_otsu
from skimage.color import rgb2gray
from skimage import img_as_ubyte
from skimage.morphology import binary_closing, disk
from skimage.filters import gaussian
import numpy as np

img = cv2.imread(input_path)
img_gray = rgb2gray(img)

thresh_otsu = threshold_otsu(img_gray)
otsu = img_gray >= thresh_otsu

close_otsu = binary_closing(otsu)
cv_close_otsu = img_as_ubyte(close_otsu)

cnts = cv2.findContours(cv_close_otsu, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2]
cnt = sorted(cnts, key=cv2.contourArea, reverse=True)[0]

rbox = cv2.minAreaRect(cnt)
pts = cv2.boxPoints(rbox).astype(np.int32)
cv2.drawContours(img, [pts], -1, (0, 255, 0), 1, cv2.LINE_AA)

cv2.imshow('Contours', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [199]:
width = 210
height = 297
area = width * height

k = (cv2.contourArea(pts) / area) ** 0.5
k

4.059853619989857

In [200]:
from dataclasses import dataclass

@dataclass
class ContourWithArea:
    """
    Object for contour and area data
    """
    contour: np.array
    area: float
        
ct= [ContourWithArea((i / k).astype(int), cv2.contourArea((i / k).astype(int))) for i in contours]

In [201]:
max_ct_ar = max([i.area for i in ct])
filter_area = 50
filtered_arr = [i.area > filter_area for i in ct]
ct = np.array(ct)
filtered_ct = ct[filtered_arr]

In [202]:
[i.area for i in filtered_ct]

[2396.0, 7257.0]

In [203]:
len(filtered_ct)

2

In [204]:
cv2.drawContours(output, [i.contour for i in filtered_ct], -1, (0, 255, 0), 3)

cv2.imshow('Contours', output)
cv2.waitKey(0)
cv2.destroyAllWindows()